In [5]:
import json
import tweepy
import pandas as pd 
import time

def loadKeys(key_file):
    with open('keys.json') as f:
        key_dict = json.load(f)
    return key_dict['api_key'], key_dict['api_secret'], key_dict['token'], key_dict['token_secret']

KEY_FILE = 'keys.json'
api_key, api_secret, token, token_secret = loadKeys(KEY_FILE)
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(token, token_secret)
api = tweepy.API(auth)

In [4]:
%%time
all_connections = pd.DataFrame()
n = 0

SLEEP_TIME = 60 * 15
def getFollowing(api, root_user, no_of_following):
    primary_following = []
    try:
        for user in tweepy.Cursor(api.friends, screen_name = root_user).items(no_of_following):
            primary_following.append((root_user, user.screen_name))

    except tweepy.RateLimitError:
        time.sleep(SLEEP_TIME) #sleep and make failed request again
        getFollowing(api, root_user, no_of_following)

    return primary_following

while len(all_connections.index) < 5000:
    if n == 0:
        user = 'WSJ'
    else:
        user = all_connections[~all_connections.iloc[:,1].isin(all_connections.iloc[:,0])].iloc[0,1]

    current_following = getFollowing(api, user, 1000)
    all_connections.append(current_following, ignore_index=True)

all_connections.to_csv('Connections_0.csv')

NameError: name 'time' is not defined